
# Install fastai dependencies

In [ ]:
pip install fastai

In [ ]:
# Import required libraries from fastai and other necessary modules
from fastai.vision.all import *
# from google.colab import drive
from pathlib import Path
import os

# Download Custom Classification Data From Roboflow

In [ ]:
# Define the path to your dataset
dataset_path = './dataset'

# Load the dataset
# Ensure your dataset has '0' and '1' directories representing classes
data = ImageDataLoaders.from_folder(
    dataset_path,
    valid_pct=0.2,
    item_tfms=Resize(224),
    batch_tfms=[*aug_transforms(), Normalize.from_stats(*imagenet_stats)]
)

# Check data classes
print(data.vocab)

# Visualize a batch of the dataset
data.show_batch(max_n=16, figsize=(7, 8))

# Set up fastai Resnet model

In [ ]:
# Create and train the model
learn = vision_learner(data, resnet34, metrics=error_rate)

# Print network layers
print(learn.model)

# Train Custom Resnet Image Classifier

In [ ]:

import torch
import gc

# Callbacks for early stopping and saving the best model
early_stop = EarlyStoppingCallback(monitor='error_rate', patience=20)
save_best_model = SaveModelCallback(monitor='error_rate', fname='resnet34')


torch.cuda.empty_cache()
gc.collect()
# def clear_gpu_memory():
#     torch.cuda.empty_cache()
#     gc.collect()
#     del variables

# Use CUDA if available, otherwise use CPU
if torch.cuda.is_available():
    # clear_gpu_memory()
    learn.dls.cuda()
    print("Using GPU (CUDA)")
    learn.fit_one_cycle(10, cbs=[early_stop, save_best_model])
else:
    learn.dls.cpu()
    print("Using CPU")

# Train with frozen layers

In [ ]:
# Load best model from frozen training
learn.load('resnet34')

# Unfreeze the model
learn.unfreeze()

# Function to find the appropriate learning rate
def find_appropriate_lr(model:Learner, lr_diff:int = 15, loss_threshold:float = .05, adjust_value:float = 1, plot:bool = False) -> float:
    # Run the Learning Rate Finder
    model.lr_find()

    # Get loss values and their corresponding gradients, and get lr values
    losses = np.array(model.recorder.losses)
    min_loss_index = np.argmin(losses)

    # Get learning rates
    lrs = model.recorder.lrs

    # Return the learning rate that produces the minimum loss divided by 10
    return lrs[min_loss_index] / 10


In [ ]:
import torch

# Find the optimal learning rate
optimal_lr = find_appropriate_lr(learn)

# Fine-tune the model with the optimal learning rate
learn.unfreeze()
learn.fit_one_cycle(100, lr_max=slice(optimal_lr/10, optimal_lr), cbs=[early_stop, save_best_model])

# Load the best model after fine-tuning
learn.load('resnet34')

# Assuming your model is named 'model'
# Assuming your optimizer is named 'optimizer'
# Assuming you want to save the model to a file named 'model.pth'

# Save the model and optimizer state
torch.save({
    'optimizer_state_dict':learn.model.state_dict()
}, 'dataset/models/resnet.pth')

# Evaluate Classifier Performance

In [ ]:
# Evaluate the classifier performance
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

# print performance matrix
interp.print_classification_report()

In [ ]:
interp.plot_top_losses(9, figsize=(15, 15))

In [ ]:
from fastai.vision.all import PILImage

# Run inference on test images
import glob
from IPython.display import Image, display

image_paths = glob.glob('./dataset/0/*.jpg')
print(image_paths)

sumC = 0
total = 0
accuracy = 0

for image_path in image_paths:
    img = PILImage.create(image_path)
    prediction = learn.predict(img)
    if prediction[0] == '1':
        sumC += 1

total = len(image_paths)
accuracy = sumC/total * 100
print("Total: ", total)
print("Correct: ", sumC)
print("Accuracy: ", accuracy)



# Save custom classification model for future use

In [ ]:
# Save custom classification model for future use
!ls models/

# Download the model file
from google.colab import files
files.download('./models/best_resnet34.pth')